<a href="https://colab.research.google.com/github/seriamedard/ASL_ResNet/blob/main/ASL_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import tensorflow as tf